In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Acute_Myeloid_Leukemia"
cohort = "GSE99612"

# Input paths
in_trait_dir = "../../input/GEO/Acute_Myeloid_Leukemia"
in_cohort_dir = "../../input/GEO/Acute_Myeloid_Leukemia/GSE99612"

# Output paths
out_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/GSE99612.csv"
out_gene_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/gene_data/GSE99612.csv"
out_clinical_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/clinical_data/GSE99612.csv"
json_path = "../../output/preprocess/Acute_Myeloid_Leukemia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"The effect of dietary fibre exposure on gene expression profiles in Caco-2 and THP-1 cells"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell line: Caco-2', 'cell line: THP-1'], 1: ['Sex: male', 'cell type: macrophage'], 2: ['treatment: medium', 'treatment: Novelose 500 ug/ml', 'treatment: Inulin-chicory 500 ug/ml', 'treatment: Resistant starch corn 500 ug/ml', 'treatment: Sugar beet pectin 500 ug/ml', 'treatment: Beta-glucan oat medium viscosity 500 ug/ml', 'treatment: GOS 500 ug/ml', 'treatment: LPS 11.85 pg/ml', 'Sex: male'], 3: ['tumor origin: Caucasian colon adenocarcinoma', 'patient age: 1 year infant'], 4: ['passage number: 30-60', 'tumor origin: acute monocytic leukemia'], 5: ['days of differentiation on tranwells: 21', 'treatment: medium', 'treatment: LPS 11.85 pg/ml', 'treatment: Novelose 500 ug/ml', 'treatment: I

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Dict, Any, Optional
import numpy as np

# 1. Gene Expression Data Availability
# Based on the background information, this appears to be a cell line experiment comparing
# Caco-2 and THP-1 cells with various treatments. While it does contain gene expression data,
# it's not suitable for our study on human AML patients.
is_gene_available = True  # The dataset likely contains gene expression data

# 2. Variable Availability and Data Type Conversion
# This dataset doesn't contain patient-level clinical data about AML.
# It's comparing different cell lines with different treatments.

# The dataset doesn't contain usable trait data for our purposes (AML vs non-AML in humans)
trait_row = None  # No suitable trait data for human patients

# Age data isn't patient age but refers to the original cell line source
age_row = None  # No suitable age data for human patients

# Sex data doesn't represent individual patients
gender_row = None  # No suitable gender data for human patients

# No need to define conversion functions since we won't use them

# 3. Save Metadata
# Since this is a cell line experiment, not patient data, it's not suitable for our study
is_trait_available = trait_row is not None  # This will be False

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this step
if trait_row is not None:
    # This code won't execute since trait_row is None
    try:
        clinical_data = pd.read_csv(f"{in_cohort_dir}/clinical_data.csv", index_col=0)
        
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=lambda x: None,  # Placeholder since we won't use it
            age_row=age_row,
            convert_age=None,
            gender_row=gender_row,
            convert_gender=None
        )
        
        # Preview the DataFrame
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Create output directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the DataFrame to CSV
        selected_clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error during clinical feature extraction: {e}")


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['7892501', '7892502', '7892503', '7892504', '7892505', '7892506',
       '7892507', '7892508', '7892509', '7892510', '7892511', '7892512',
       '7892513', '7892514', '7892515', '7892516', '7892517', '7892518',
       '7892519', '7892520'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Review of gene identifiers in the gene expression data
# The identifiers appear to be numerical codes (like 7892501, 7892502, etc.)
# These are likely probe IDs rather than standard human gene symbols
# Human gene symbols would be alphanumeric (like BRCA1, TP53, etc.)
# Therefore, these identifiers will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001005240,NM_001004195,NM_001005484,BC136848,BC136907', 'BC118988,AL137655', 'NM_001005277,NM_001005221,NM_001005224,NM_001005504,BC137547'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': [53049.0, 63015.0, 69091.0, 334129.0, 367659.0], 'RANGE_STOP': [54936.0, 63887.0, 70008.0, 334296.0, 368597.0], 'total_probes': [7.0, 31.0, 24.0, 6.0, 36.0], 'gene_assignment': ['---', '---', 'NM_001005240 // OR4F17 // olfactory receptor, family 4, subfamily F, member 17 // 19p13.3 // 81099 /// NM_001004195 // OR4F4 // olfactory receptor, family 4, subfamily F, member 4 // 15q26.3 // 26682 /// NM_001005484 // OR4F5 //

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for probe IDs and gene symbols
probe_col = 'ID'  # This contains the probe identifiers like '7896736'
gene_col = 'gene_assignment'  # This contains gene symbol information

# 2. Get gene mapping using the function from the library
mapping_df = get_gene_mapping(gene_annotation, probe_col, gene_col)

# Print a sample of the mapping to verify structure
print("Gene mapping sample (first 5 rows):")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print the shape and first few rows of the resulting gene expression dataframe
print("\nGene expression data shape after mapping:", gene_data.shape)
print("\nFirst 5 genes and expression values:")
print(gene_data.head())

# Normalize gene symbols to ensure consistency and handle synonyms
gene_data = normalize_gene_symbols_in_index(gene_data)

# Print the shape after normalization
print("\nGene expression data shape after normalization:", gene_data.shape)

# Save gene data to output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Gene mapping sample (first 5 rows):
        ID                                               Gene
0  7896736                                                ---
1  7896738                                                ---
2  7896740  NM_001005240 // OR4F17 // olfactory receptor, ...
3  7896742  ENST00000388975 // SEPT14 // septin 14 // 7p11...
4  7896744  NM_001005277 // OR4F16 // olfactory receptor, ...



Gene expression data shape after mapping: (56391, 48)

First 5 genes and expression values:
      GSM2648543  GSM2648544  GSM2648545  GSM2648546  GSM2648547  GSM2648548  \
Gene                                                                           
A-     38.716029   37.128762   37.511108   37.623609   38.177840   38.209748   
A-2     2.658533    2.637381    2.639023    2.650405    2.663684    2.659015   
A-52    3.651613    3.645483    3.609023    3.648133    3.642320    3.644010   
A-E     0.482038    0.478818    0.496260    0.467780    0.487119    0.477848   
A-I    11.601787   11.668377   11.653517   11.683830   11.639600   11.745367   

      GSM2648549  GSM2648550  GSM2648551  GSM2648552  ...  GSM2648581  \
Gene                                                  ...               
A-     37.516400   37.700195   38.048053   38.039465  ...   34.037491   
A-2     2.636998    2.577943    2.640939    2.644001  ...    2.159168   
A-52    3.653870    3.661173    3.648297    3.637170  


Gene expression data saved to ../../output/preprocess/Acute_Myeloid_Leukemia/gene_data/GSE99612.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Gene data was already normalized and saved in Step 6
# No need to normalize again as it was done in Step 6

# 2-6. Since there's no clinical data available for this dataset (trait_row was None in Step 2),
# we can't link clinical and genetic data
# Instead, we should finalize the cohort information to reflect this limitation

# Get a small sample of the normalized gene data for the validation function
if 'normalized_gene_data' not in locals():
    # Load the saved gene data if not already in memory
    try:
        normalized_gene_data = pd.read_csv(out_gene_data_file, index_col=0)
    except:
        normalized_gene_data = gene_data  # Use the gene_data from previous step if file not found

# Create a minimal dataframe with the gene data structure and add a dummy trait column
minimal_df = pd.DataFrame(index=normalized_gene_data.columns)
minimal_df[trait] = None  # Add trait column with null values

# Note for the validation function explaining why this dataset isn't usable
note = "This dataset contains gene expression from cell lines (Caco-2 and THP-1) with various treatments, not human patient data for AML studies."

# Final validation - mark as not usable for trait analysis
is_trait_available = False
is_gene_available = True
is_biased = False  # Explicitly set to False since there's no trait data to evaluate bias

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=minimal_df,
    note=note
)

# We do not save linked_data to out_data_file because this dataset is not usable for the study
if is_usable:
    print("WARNING: This dataset was unexpectedly marked as usable, which conflicts with previous findings.")
else:
    print("Dataset correctly identified as not usable for human AML trait-gene association study.")

Abnormality detected in the cohort: GSE99612. Preprocessing failed.
Dataset correctly identified as not usable for human AML trait-gene association study.
